# Lesson 3: Building an Agent Reasoning Loop

## Setup

In [2]:
from helper import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()

In [3]:
import nest_asyncio
nest_asyncio.apply()

## Setup the Query Tools

In [4]:
from utils import get_doc_tools

vector_tool, summary_tool = get_doc_tools("docs/metagpt.pdf", "metagpt")

## Setup Function Calling Agent

In [5]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0)

* In LlamaIndex, agent contains two main components
    * AgentRunner: Overall task dispatcher, which is responsible for creating and orchestrating tasks, and return the final response to the user.
    * AgentWorker: Executing the next step

In [6]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

# define agentWorker and passing the two sets of tools to the worker
agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool], 
    llm=llm, 
    verbose=True
)

# agentRunner offers the interface for us to query the agent
agent = AgentRunner(agent_worker)

In [7]:
# We can see that the agent breaks the task into steps
response = agent.query(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

Added user message to memory: Tell me about the agent roles in MetaGPT, and then how they communicate with each other.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "agent roles in MetaGPT"}
=== Function Output ===
The agent roles in MetaGPT include Product Manager, Architect, Project Manager, Engineer, and QA Engineer. Each role has specific responsibilities tailored to their expertise within the collaborative framework, contributing to the breakdown of complex tasks into specialized components for efficient problem-solving and software development.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "how agents communicate with each other in MetaGPT"}
=== Function Output ===
Agents in MetaGPT communicate with each other through structured communication interfaces, a shared message pool, and a publish-subscribe mechanism. They utilize role-specific interests to extract relevant information and manage task-related 

In [9]:
# inspect the source_nodes of the response
print(response.source_nodes[1].get_content(metadata_mode="all"))

page_label: 2
file_name: metagpt.pdf
file_path: metagpt.pdf
file_type: application/pdf
file_size: 16911937
creation_date: 2024-05-15
last_modified_date: 2024-05-15

Preprint
Figure 1: The software development SOPs between MetaGPT and real-world human teams.
In software engineering, SOPs promote collaboration among various roles. MetaGPT showcases
its ability to decompose complex tasks into specific actionable procedures assigned to various roles
(e.g., Product Manager, Architect, Engineer, etc.).
documents, design artifacts, flowcharts, and interface specifications. The use of intermediate struc-
tured outputs significantly increases the success rate of target code generation. Because it helps
maintain consistency in communication, minimizing ambiguities and errors during collaboration.
More graphically, in a company simulated by MetaGPT, all employees follow a strict and stream-
lined workflow, and all their handovers must comply with certain established standards. This reduces
the ri

In [10]:
# The agent is able to maintain a conversation about its reasoning and responses
response = agent.chat(
    "Tell me about the evaluation datasets used."
)

Added user message to memory: Tell me about the evaluation datasets used.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "evaluation datasets in MetaGPT"}
=== Function Output ===
The evaluation datasets used in MetaGPT include HumanEval and MBPP. These datasets cover a wide range of programming tasks, including handwritten programming tasks and Python tasks. Additionally, the SoftwareDev dataset is part of the evaluation datasets in MetaGPT, consisting of diverse software development tasks like creating games, image processing algorithms, and data visualization. These datasets are utilized to assess the performance of MetaGPT in code generation tasks and compare its effectiveness against other benchmarks.
=== LLM Response ===
The evaluation datasets used in MetaGPT include HumanEval, MBPP, and the SoftwareDev dataset. These datasets cover various programming tasks, including handwritten programming tasks, Python tasks, and diverse software developm

In [11]:
response = agent.chat("Tell me the results over one of the above datasets.")

Added user message to memory: Tell me the results over one of the above datasets.
=== Calling Function ===
Calling function: vector_tool_metagpt with args: {"query": "results over HumanEval dataset", "page_numbers": ["7"]}
=== Function Output ===
MetaGPT achieved 85.9% and 87.7% Pass rates over the HumanEval dataset.
=== LLM Response ===
MetaGPT achieved pass rates of 85.9% and 87.7% over the HumanEval dataset.


## Lower-Level: Debuggability and Control

The key benefits for agent control at a granular manner

* Decoupling of task creation and execution
    * Users gain the flexibility to schedule task execution according to their needs.
* Enhanced debuggability
    * Offers deeper insights into each step of the execution process, improving troubleshooting capabilities.
* Steerability: Allows users to directly modify intermediate steps and incorporate human feedback for refined control. For example, we could create an async queue that listen to and incorporate human inputs throughout the agent execution. 

In [8]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool], 
    llm=llm, 
    system_prompt=""" \
        You are an agent designed to answer queries over a set of given papers.
        Please always use the tools provided to answer a question. Do not rely on prior knowledge.\

        """,
    verbose=True
)
agent = AgentRunner(agent_worker)

In [9]:
# this will return a task object which contains the input and additional state in the task object
task = agent.create_task(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

In [10]:
# the agent will execute one step of the task and return the result
step_output = agent.run_step(task.task_id)

Added user message to memory: Tell me about the agent roles in MetaGPT, and then how they communicate with each other.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "agent roles in MetaGPT"}
=== Function Output ===
The agent roles in MetaGPT include Product Manager, Architect, Project Manager, Engineer, and QA Engineer. Each role has specific responsibilities and skills tailored to their expertise within the collaborative framework of MetaGPT. The Product Manager analyzes user requirements and formulates a detailed PRD, the Architect translates requirements into system design components, the Project Manager handles task distribution, Engineers execute designated classes and functions, and the QA Engineer formulates test cases to ensure code quality. These roles work together to contribute to the software development process within the MetaGPT framework.


In [11]:
completed_steps = agent.get_completed_steps(task.task_id)
print(f"Num completed for task {task.task_id}: {len(completed_steps)}")
print(completed_steps[0].output.sources[0].raw_output)

Num completed for task bf2a2865-b6b9-455b-993d-972e0445e3c2: 1
The agent roles in MetaGPT include Product Manager, Architect, Project Manager, Engineer, and QA Engineer. Each role has specific responsibilities and skills tailored to their expertise within the collaborative framework of MetaGPT. The Product Manager analyzes user requirements and formulates a detailed PRD, the Architect translates requirements into system design components, the Project Manager handles task distribution, Engineers execute designated classes and functions, and the QA Engineer formulates test cases to ensure code quality. These roles work together to contribute to the software development process within the MetaGPT framework.


In [12]:
upcoming_steps = agent.get_upcoming_steps(task.task_id)
print(f"Num upcoming steps for task {task.task_id}: {len(upcoming_steps)}")
upcoming_steps[0]

Num upcoming steps for task bf2a2865-b6b9-455b-993d-972e0445e3c2: 1


TaskStep(task_id='bf2a2865-b6b9-455b-993d-972e0445e3c2', step_id='ec42e0cf-4841-4500-906d-a3e64ca14cb3', input=None, step_state={}, next_steps={}, prev_steps={}, is_ready=True)

In [13]:
upcoming_steps[0].is_ready

True

In [14]:
#  injecting a step through user input
step_output = agent.run_step(
    task.task_id, input="What about how agents share information?"
)

Added user message to memory: What about how agents share information?
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "how agents share information in MetaGPT"}
=== Function Output ===
Agents in MetaGPT share information through a structured communication protocol that includes a shared message pool where agents publish structured messages. They can also subscribe to relevant messages based on their profiles. This protocol enhances role communication efficiency by allowing agents to obtain directional information from other roles and public information from the environment. Additionally, MetaGPT implements a publish-subscribe mechanism that enables agents to exchange information effectively within the multi-agent system.


In [38]:
# run the next step
step_output = agent.run_step(task.task_id)
# check whether this is the last step
print(step_output.is_last)

=== LLM Response ===
Agents in MetaGPT share information through a structured communication protocol that includes a shared message pool. This pool allows agents to publish structured messages and subscribe to relevant information based on their profiles. By utilizing this communication interface, agents can efficiently exchange directional information from other roles and public information from the environment. Additionally, MetaGPT implements a publish-subscribe mechanism that enables agents to access important observations stored in the message pool, which can trigger actions or aid in completing tasks effectively. The shared message pool and subscription mechanism in MetaGPT facilitate the exchange of information among agents within the simulated software company framework.
True


In [40]:
response = agent.finalize_response(task.task_id)

In [41]:
print(str(response))

assistant: Agents in MetaGPT share information through a structured communication protocol that includes a shared message pool. This pool allows agents to publish structured messages and subscribe to relevant information based on their profiles. By utilizing this communication interface, agents can efficiently exchange directional information from other roles and public information from the environment. Additionally, MetaGPT implements a publish-subscribe mechanism that enables agents to access important observations stored in the message pool, which can trigger actions or aid in completing tasks effectively. The shared message pool and subscription mechanism in MetaGPT facilitate the exchange of information among agents within the simulated software company framework.


In [42]:
# overview of all the completed steps
agent.get_completed_steps(task.task_id)

[TaskStepOutput(output=AgentChatResponse(response='assistant: None', sources=[ToolOutput(content='The agent roles in MetaGPT include Product Manager, Architect, Project Manager, Engineer, and QA Engineer. The Product Manager is responsible for generating the Product Requirement Document (PRD) outlining goals, user stories, competitive analysis, and requirement analysis. The Architect designs the system based on the PRD. The Project Manager distributes tasks, the Engineer executes code, and the QA Engineer formulates test cases to ensure code quality. Each role has specific responsibilities tailored to different aspects of the collaborative software development process within the MetaGPT framework.', tool_name='summary_tool_metagpt', raw_input={'input': 'agent roles in MetaGPT'}, raw_output=Response(response='The agent roles in MetaGPT include Product Manager, Architect, Project Manager, Engineer, and QA Engineer. The Product Manager is responsible for generating the Product Requirement